In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ca = pd.read_csv('../data/Initiated Building Permits (CA Tasks) 3yr.csv',
                 parse_dates=['Date_Initiated','Date_Issued','Task_Complete_Date']) 
fr = pd.read_csv('../data/Initiated Building Permits (FR Tasks) 3yr.csv',
                 parse_dates=['Date_Initiated','Date_Issued','Task_Complete_Date']) 
pl = pd.read_csv('../data/Initiated Building Permits (PL Tasks) 3yr.csv',
                 parse_dates=['Date_Initiated','Date_Issued','Task_Complete_Date']) 
pw = pd.read_csv('../data/Initiated Building Permits (PW Tasks) 3yr.csv', 
                 parse_dates=['Date_Initiated','Date_Issued','Task_Complete_Date']) 
sw = pd.read_csv('../data/Initiated Building Permits (SW Tasks) 3yr.csv', 
                 parse_dates=['Date_Initiated','Date_Issued','Task_Complete_Date']) 
ws = pd.read_csv('../data/Initiated Building Permits (WS Tasks) 3yr.csv', 
                 parse_dates=['Date_Initiated','Date_Issued','Task_Complete_Date']) 
u_o = pd.read_csv('../data/Initiated Building Permits 3yr (Temp-Final U_O).csv', 
                  parse_dates=['Date_Initiated','Date_Issued','Temp U&O Complete Date','Final U&O Complete Date']) 

C:\Users\pfbab\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
department = [['Codes(CA)', ca],
              ['Fire Marshal(FR)', fr],
              ['Planning(PL)', pl],
              ['Public Works(PW)', pw],
              ['Storm Water (SW)', sw],
              ['Water Sewer(WS)', ws] ]
df_list = list()


,CASE_TYPE,CASE_TYPE_DESC,DEPARTMENT
7,CACA,Building Commercial - Addition,Codes(CA)
6,CACA,Building Commercial - Addition,Fire Marshal(FR)
8,CACA,Building Commercial - Addition,Planning(PL)
7,CACA,Building Commercial - Addition,Public Works(PW)
6,CACA,Building Commercial - Addition,Storm Water (SW)
...,...,...,...
9,CART,Building Residential - Tenant Finish Out,Fire Marshal(FR)
18,CART,Building Residential - Tenant Finish Out,Planning(PL)
14,CART,Building Residential - Tenant Finish Out,Public Works(PW)
20,CART,Building Residential - Tenant Finish Out,Storm Water (SW)


In [ ]:
#from Patrick
for a,b in department:
    ct = b[['CASE_TYPE', 'CASE_TYPE_DESC']].copy()
    ct['DEPARTMENT'] = a
    ct_clean = ct.value_counts().reset_index().drop(columns=[0])
    df_list.append(ct_clean)
ct_df = pd.concat(df_list).sort_values(by =['CASE_TYPE', 'DEPARTMENT'])
ct_df

In [4]:
list_dept = {'fr':fr,'pl':pl,'pw':pw,'sw':sw,'ws':ws,'ca':ca}
convert_dict = {'CASE_TYPE': 'category',
                'SUB_TYPE': 'category',
                'STATUS_CODE': 'category',
                'RESULT_CODE': 'category'
               }
all_depts_app = pd.DataFrame()
all_depts_inspect = pd.DataFrame()
task_filter = {}
all_pivot = pd.DataFrame()

In [5]:
for key in list_dept:
    df = list_dept[key]
    df['dept'] = str(key)
    df = df[df['PERMIT #'].astype(str).str.isnumeric()] # T&D filter
    df = df.astype(convert_dict)
    df['deltatask'] = df['Task_Complete_Date'] - df['Date_Initiated'] #create column for task length
    df = df.astype({'deltatask':"timedelta64[D]"}) #make timedelta work for pivot table
    # find tasks completed after permit issued
    df['postissuedtask'] = df['Task_Complete_Date'] > df['Date_Issued']
    post_taskcode = df[['TASK_DESC','postissuedtask']].groupby(['TASK_DESC']).agg(['sum', 'count'])
    filter_posttaskcode = post_taskcode[post_taskcode.iloc[:,0] >= post_taskcode.iloc[:,1]*.95].reset_index()
    #create list of tasks for application and inspection 
    df_inspect = df[df['TASK_DESC'].isin(filter_posttaskcode.TASK_DESC)]
    df_app = df[~df['TASK_DESC'].isin(filter_posttaskcode.TASK_DESC)]
    #keep only the last task completed
    df_last = df_app.sort_values(by=['Task_Complete_Date','RESULT_DESC'], ascending=[False,True]).drop_duplicates(subset=['PERMIT #', 'TASK_CODE'], keep='first')
    #pivot data to aggregate
    df_pivot = pd.pivot_table(df_last, values='deltatask', index=['CASE_TYPE'], aggfunc=[np.mean, np.median, np.min, np.max, np.std, np.var], margins = True)
    df_pivot['dept'] = str(key)
    #save all the steps
    all_depts_app = all_depts_app.append(df_last)
    all_depts_inspect = all_depts_inspect.append(df_inspect)
    task_filter.update({str(key):filter_posttaskcode['TASK_DESC'].to_list()})
    all_pivot = all_pivot.append(df_pivot)
    print(key)
    
    

fr
pl
pw
sw
ws
ca


In [6]:
task_filter

{'fr': ['CA - U&O Fire Sprinkler Final Approval',
  'Fire Alarm Review On Bldg App',
  'Fire Sprinkler RTC Review - CA Return to Customer',
  'Fire Sprinkler Review On Bldg App',
  'Fire Sprinkler Review on Alarm App',
  'Life Safety Rough In',
  'U&O Alarm Fire Sprinkler Final Approval',
  'U&O Fire Alarm System Final Approval - CA',
  'U&O Fire Alarm System Final Approval - CAAF',
  'U&O Life Safety Final Approval',
  'U&O Underground Work Final Approval - CA',
  '[B]  Fire Alarm System Review',
  '[B]  Fire Sprinkler Review On Bldg App',
  '[B] Fire Alarm Review On Bldg App',
  '[B] Fire Underground Work Review On Bldg App'],
 'pl': [],
 'pw': ['CA - U&O Public Works Final Approval',
  'U&O PW ROW FA - CA Final Approval',
  'U&O PW Ramp & Curb Cut FA - CA Final Approval',
  'U&O PW Ramp RTC & Cut - CA Return to Customer',
  'U&O PW Sidewalk FA - CA Final Approval',
  'U&O PW Solid Waste FA - CA Final Approval'],
 'sw': ['SWGR U&O Sign-off', 'U&O SW Flood Plain Final Approval - CA'],

In [7]:
all_depts_app.RESULT_DESC.value_counts()

Approved                                   128918
Approved with Comments                      48947
No sign-off needed on this project          41071
Issued                                      27004
OK to Pay Permit Fee                        25649
Not Applicable                              25253
Yes                                         12386
No                                           5128
Sidewalks Required                           4689
Paper plans received                         3309
Electronic plans received                    3135
No Capital Project Planned                   1812
Sidewalks Required - In Lieu Elligble        1368
Sidewalks not required                       1279
Allowed and elected to pay in lieu           1217
Elected/Required to construct sidewalks       290
Plans received via email as a PDF             226
Granted with Conditions                       119
BZA Variance Applied for                       85
Zoning Waiver Requested                        77


In [8]:
alltemppivot = pd.pivot_table(all_depts_app, values='deltatask', index=['CASE_TYPE'], aggfunc=[np.mean, np.median, np.min, np.max, np.std, np.var], margins = True)
alltemppivot['dept'] = str('all')
all_pivot = all_pivot.append(alltemppivot)
all_pivot = all_pivot.reset_index()

In [9]:
taskfilterdf = pd.DataFrame({ key:pd.Series(value) for key, value in task_filter.items() })

<ipython-input-9-a55de755209d>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  taskfilterdf = pd.DataFrame({ key:pd.Series(value) for key, value in task_filter.items() })


In [14]:
all_pivot

,CASE_TYPE,mean,median,amin,amax,std,var,dept
,,deltatask,deltatask,deltatask,deltatask,deltatask,deltatask,
0,CACA,59.356234,30.0,0.0,529.0,80.371588,6459.592161,fr
1,CACF,34.920635,15.0,0.0,385.0,58.886835,3467.659286,fr
2,CACG,58.962264,48.0,16.0,198.0,44.781075,2005.344702,fr
3,CACH,95.737885,71.0,0.0,604.0,94.584601,8946.246818,fr
4,CACJ,27.380282,1.0,0.0,152.0,47.071486,2215.724748,fr
...,...,...,...,...,...,...,...,...
166,CARP,7.824885,0.0,0.0,124.0,16.301907,265.752174,all
167,CARR,5.649149,0.0,-30.0,574.0,18.484825,341.688740,all
168,CARS,19.288732,10.0,0.0,158.0,26.251133,689.121988,all


In [12]:
department = [['Codes(CA)', ca],
              ['Fire Marshal(FR)', fr],
              ['Planning(PL)', pl],
              ['Public Works(PW)', pw],
              ['Storm Water (SW)', sw],
              ['Water Sewer(WS)', ws] ]
df_list = list()
for a,b in department:
    ct = b[['CASE_TYPE', 'CASE_TYPE_DESC']].copy()
    ct['DEPARTMENT'] = a
    ct_clean = ct.value_counts().reset_index().drop(columns=[0])
    df_list.append(ct_clean)
ct_df = pd.concat(df_list).sort_values(by =['CASE_TYPE', 'DEPARTMENT'])
all_pivot_label = pd.merge(ct_df,all_pivot, how='right' , on='CASE_TYPE')

C:\Users\pfbab\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\pfbab\anaconda3\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [13]:
all_pivot_label.to_csv('../data/allpivot.csv', index=False)
all_depts_app.to_csv('../data/all_depts_app.csv', index=False)
all_depts_inspect.to_csv('../data/all_depts_inspect.csv', index=False)
taskfilterdf.to_csv('../data/taskfilter.csv', index=False)

In [15]:
all_pivot_label

,CASE_TYPE,CASE_TYPE_DESC,DEPARTMENT,"(mean, deltatask)","(median, deltatask)","(amin, deltatask)","(amax, deltatask)","(std, deltatask)","(var, deltatask)","(dept, )"
0,CACA,Building Commercial - Addition,Codes(CA),59.356234,30.0,0.0,529.0,80.371588,6459.592161,fr
1,CACA,Building Commercial - Addition,Fire Marshal(FR),59.356234,30.0,0.0,529.0,80.371588,6459.592161,fr
2,CACA,Building Commercial - Addition,Planning(PL),59.356234,30.0,0.0,529.0,80.371588,6459.592161,fr
3,CACA,Building Commercial - Addition,Public Works(PW),59.356234,30.0,0.0,529.0,80.371588,6459.592161,fr
4,CACA,Building Commercial - Addition,Storm Water (SW),59.356234,30.0,0.0,529.0,80.371588,6459.592161,fr
...,...,...,...,...,...,...,...,...,...,...
962,CART,Building Residential - Tenant Finish Out,Planning(PL),66.496190,27.0,0.0,474.0,87.116068,7589.209232,all
963,CART,Building Residential - Tenant Finish Out,Public Works(PW),66.496190,27.0,0.0,474.0,87.116068,7589.209232,all
964,CART,Building Residential - Tenant Finish Out,Storm Water (SW),66.496190,27.0,0.0,474.0,87.116068,7589.209232,all
965,CART,Building Residential - Tenant Finish Out,Water Sewer(WS),66.496190,27.0,0.0,474.0,87.116068,7589.209232,all


,PERMIT #,CASE_TYPE,CASE_TYPE_DESC,SUB_TYPE,SUB_TYPE_DESC,CONSTRUCTION COST,Square Footage,STATUS_CODE,Date_Initiated,Date_Issued,TASK_CODE,TASK_DESC,RESULT_CODE,RESULT_DESC,Task_Complete_Date,dept,deltatask,postissuedtask
2587,2018000351,CACR,Building Commercial - Rehab,CAD02B016,"General Office, Professional Services",80000.00,0.0,DONE,2018-01-03,2018-01-03,FRULIFESAF,U&O Life Safety Final Approval,APPROVED,Approved,2018-02-09,fr,37.0,True
2595,2018000607,CACR,Building Commercial - Rehab,CAD02B016,"General Office, Professional Services",176925.00,0.0,DONE,2018-01-04,2018-01-04,FRBSPRINK,[B] Fire Sprinkler Review On Bldg App,APPROVED,Approved,2018-02-22,fr,49.0,True
2678,2018001141,CACR,Building Commercial - Rehab,CAD02B016,"General Office, Professional Services",95000.00,0.0,DONE,2018-01-08,2018-01-10,FRULIFESAF,U&O Life Safety Final Approval,APPROVED,Approved,2018-03-07,fr,58.0,True
2679,2018001141,CACR,Building Commercial - Rehab,CAD02B016,"General Office, Professional Services",95000.00,0.0,DONE,2018-01-08,2018-01-10,FRBSPRINK,[B] Fire Sprinkler Review On Bldg App,IGNORE,No sign-off needed on this project,2018-03-05,fr,56.0,True
2682,2018001141,CACR,Building Commercial - Rehab,CAD02B016,"General Office, Professional Services",95000.00,0.0,DONE,2018-01-08,2018-01-10,FRUSPRINKL,CA - U&O Fire Sprinkler Final Approval,APPROVED,Approved,2018-03-05,fr,56.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284314,2020039853,CARN,Building Residential - New,CAA01R301,Single Family Residence,822656.24,8068.0,ISSUE,2020-06-26,2021-05-11,CABFRAME,Building Framing,CANCEL,Cancel,2021-05-14,ca,322.0,True
284315,2020039853,CARN,Building Residential - New,CAA01R301,Single Family Residence,822656.24,8068.0,ISSUE,2020-06-26,2021-05-11,CABFOOT,Building Footing,REJ-NOFEE,Rejected - No Reinspection Fee Required,2021-05-25,ca,333.0,True
284364,2020047039,CACH,Building Commercial - Shell,SHELL,Commercial Shell Generic,373424.32,5198.0,ISSUE,2020-07-29,2021-05-13,CABCFOOT,Commercial Building Footing,APPROVED,Approved,2021-05-21,ca,296.0,True
284439,2020078739,CACN,Building Commercial - New,CAF24M003,"Automobile Convenience, Markets",515270.00,0.0,ISSUE,2020-12-17,2021-05-17,CABCFOOT,Commercial Building Footing,APPROVED,Approved,2021-05-21,ca,155.0,True
